In [1]:
from lobster_loader import LoadLOBSTER_resample
import os

# Get the current workspace directory
workspace_dir = r"f:\OneDrive - 7t12kn\Documents\GitHub\ViT-MARL"

# Initialize the loader for your AMZN data
loader = LoadLOBSTER_resample(
    datapath=workspace_dir,              # Base directory where AMZN files are
    atpath=workspace_dir,                # Where to save cached .npz files
    stock="AMZN",                        # Stock ticker (not used since files are in root)
    time_period="2012-06-21",            # Time period (not used since files are in root)
    n_Levels=10,                         # Number of order book levels
    type_="fixed_time",                  # Use time-based windows
    window_length=1800,                  # 30-minute windows (1800 seconds)
    window_resolution=60,                # New window every 60 seconds
    n_data_msg_per_step=100,             # 100 messages per step
    day_start=34200,                     # 9:30 AM (in seconds since midnight)
    day_end=57600                        # 4:00 PM (in seconds since midnight)
)

print(f"\nFound files:")
print(f"  Message files: {len(loader.message_files)}")
print(f"  Book files: {len(loader.book_files)}")

# Load the data (will use cached .npz if available)
print("\nLoading data...")
msgs, starts, ends, obs, max_msgs = loader.run_loading()

print(f"\nData loaded successfully!")
print(f"  Total messages: {msgs.shape}")
print(f"  Number of windows: {len(starts)}")
print(f"  Window start indices shape: {starts.shape}")
print(f"  Window end indices shape: {ends.shape}")
print(f"  Initial orderbook states shape: {obs.shape}")
print(f"  Max messages per window: {max_msgs[:5]}...")  # Show first 5

# Example: Access first window's data
print(f"\nFirst window:")
print(f"  Starts at message index: {starts[0]}")
print(f"  Ends at message index: {ends[0]}")
print(f"  Contains {ends[0] - starts[0]} messages")
print(f"  First 3 messages in window:")
print(msgs[starts[0]:starts[0]+3])

self.datapath f:\OneDrive - 7t12kn\Documents\GitHub\ViT-MARL/AMZN/2012-06-21/
found 1 message files
found 1 book files

Found files:
  Message files: 1
  Book files: 1

Loading data...
Loading cached arrays from f:\OneDrive - 7t12kn\Documents\GitHub\ViT-MARL\saved_npz/lobster_AMZN_2012-06-21_10_fixed_time_1800_60_100_34200_57600.npz

Data loaded successfully!
  Total messages: (267328, 8)
  Number of windows: 390
  Window start indices shape: (390,)
  Window end indices shape: (390,)
  Initial orderbook states shape: (390, 40)
  Max messages per window: [17217 17792 18569 19000 19844]...

First window:
  Starts at message index: 0
  Ends at message index: 17217
  Contains 17217 messages
  First 3 messages in window:
[[        1         1        21   2238100  11885113  11885113     34200
  189607669]
 [        1        -1        20   2239600   3911376   3911376     34200
  189607669]
 [        1         1       100   2237500  11534792  11534792     34200
  189607669]]


In [2]:
import numpy as np
data = np.load(r"f:\OneDrive - 7t12kn\Documents\GitHub\ViT-MARL\saved_npz\lobster_AMZN_2012-06-21_10_fixed_time_1800_60_100_34200_57600.npz", allow_pickle=True)
msgs, starts, ends, obs, max_msgs = data["msgs"], data["starts"], data["ends"], data["obs"], data["max_msgs_in_windows_arr"]

In [3]:
from gymnax_exchange.jaxlobster.data_loading import *
# from gymnax_exchange.jaxob.JaxOrderBookArrays import *
# from gymnax_exchange.jaxob.jorderbook import *
import torch
torch.cuda.is_available()

True

In [4]:
import platform
print(platform.system())

Windows


In [5]:
import os
workspace_dir = r"f:\OneDrive - 7t12kn\Documents\GitHub\ViT-MARL"
os.makedirs(os.path.join(workspace_dir, "saved_npz"), exist_ok=True)

In [6]:
print("##AI-generated placeholder")
# ═══════════════════════════════════════════════════════════════
# EXPLORING THE LOADED DATA
# ═══════════════════════════════════════════════════════════════

print("="*70)
print("DATA LOADED - Let's explore what we have:")
print("="*70)

print(f"\n1. MESSAGES ARRAY (msgs):")
print(f"   Shape: {msgs.shape}")
print(f"   → {msgs.shape[0]:,} total messages")
print(f"   → 8 features per message")
print(f"\n   First message: {msgs[0]}")
print(f"   Features: [type, direction, qty, price, trader_id, order_id, time_s, time_ns]")

print(f"\n2. EPISODE WINDOWS:")
print(f"   Number of windows: {len(starts)}")
print(f"   → This means {len(starts)} different starting points for episodes!")

print(f"\n3. WINDOW STRUCTURE:")
for i in range(min(3, len(starts))):
    start_idx = starts[i]
    end_idx = ends[i]
    n_msgs = end_idx - start_idx
    n_steps = n_msgs // 100  # Assuming 100 msgs per step
    
    # Get time
    first_msg_time = msgs[start_idx][-2]
    hours = first_msg_time // 3600
    minutes = (first_msg_time % 3600) // 60
    
    print(f"\n   Window {i}:")
    print(f"     Start time: {hours:02d}:{minutes:02d}")
    print(f"     Messages: [{start_idx:,} : {end_idx:,}] = {n_msgs:,} messages")
    print(f"     Steps: ~{n_steps} steps (at 100 msgs/step)")
    print(f"     Initial orderbook shape: {obs[i].shape}")

print(f"\n4. ORDERBOOK SNAPSHOTS:")
print(f"   Shape: {obs.shape}")
print(f"   → {obs.shape[0]} initial orderbooks (one per window)")
print(f"   → {obs.shape[1]} values = 10 levels × 2 sides × 2 (price, qty)")
print(f"\n   First orderbook snapshot:")
print(f"   {obs[0]}")

print("\n" + "="*70)
print("✓ Data exploration complete!")
print("="*70)

##AI-generated placeholder
DATA LOADED - Let's explore what we have:

1. MESSAGES ARRAY (msgs):
   Shape: (267328, 8)
   → 267,328 total messages
   → 8 features per message

   First message: [        1         1        21   2238100  11885113  11885113     34200
 189607669]
   Features: [type, direction, qty, price, trader_id, order_id, time_s, time_ns]

2. EPISODE WINDOWS:
   Number of windows: 390
   → This means 390 different starting points for episodes!

3. WINDOW STRUCTURE:

   Window 0:
     Start time: 09:30
     Messages: [0 : 17,217] = 17,217 messages
     Steps: ~172 steps (at 100 msgs/step)
     Initial orderbook shape: (40,)

   Window 1:
     Start time: 09:31
     Messages: [672 : 18,464] = 17,792 messages
     Steps: ~177 steps (at 100 msgs/step)
     Initial orderbook shape: (40,)

   Window 2:
     Start time: 09:32
     Messages: [871 : 19,440] = 18,569 messages
     Steps: ~185 steps (at 100 msgs/step)
     Initial orderbook shape: (40,)

4. ORDERBOOK SNAPSHOTS:
  

In [7]:
print("##AI-generated placeholder")
# ═══════════════════════════════════════════════════════════════
# CREATING BASELOBENV - This will use the loaded data
# ═══════════════════════════════════════════════════════════════

import jax
from gymnax_exchange.jaxen.base_env import BaseLOBEnv
from gymnax_exchange.jaxob.jaxob_config import World_EnvironmentConfig

print("="*70)
print("CREATING BASELOBENV")
print("="*70)

# Create configuration - MUST match the loader settings!
# ⚠️ KEY ISSUE: These parameters must match what you used in LoadLOBSTER_resample above!
workspace_dir = r"f:\OneDrive - 7t12kn\Documents\GitHub\ViT-MARL"

config = World_EnvironmentConfig(
    # ═══ CRITICAL: Path Configuration ═══
    dataPath=workspace_dir,              # ← WHERE YOUR CSV FILES ARE!
    alphatradePath=workspace_dir,        # ← WHERE TO SAVE CACHE FILES!
    
    # ═══ Episode Configuration ═══
    # window_selector=-1,                  # -1 = random window, 0-389 = specific window
    ep_type="fixed_time",                # Same as loader
    episode_time=1800,                   # 30 minutes
    n_data_msg_per_step=100,             # 100 messages per step
    
    # ═══ Time Window Configuration ═══
    # day_start=34200,                     # 9:30 AM (in seconds since midnight)
    # day_end=57600,                       # 4:00 PM
    start_resolution=60,                 # New window every 60s
    
    # ═══ Market Data Configuration ═══
    # book_depth=10,                       # 10 levels of order book
    stock="AMZN",                        # Stock ticker
    timePeriod="2012-06-21",             # Date
    
    # ═══ Performance Configuration ═══
    use_pickles_for_init=True            # Use cached states if available
)

# Initialize JAX random key
rng = jax.random.PRNGKey(0)
key_init, key_reset = jax.random.split(rng, 2)

print("\n📦 Creating environment...")
print("   This will:")
print("   1. Check for cached .npz (we just saw it loads instantly!)")
print("   2. Check for cached .pkl (init states)")
print("   3. If no cache: compute initial states for all windows")
print("   4. Otherwise: load from cache (fast!)")
print()

# This is where the magic happens!
env = BaseLOBEnv(cfg=config, key=key_init)

print("\n✓ Environment created!")
print(f"   Number of episode windows: {env.n_windows}")
print(f"   Messages available: {env.messages.shape}")
print(f"   Initial states cached: {env.init_states_array is not None}")

print("\n" + "="*70)

##AI-generated placeholder
CREATING BASELOBENV

📦 Creating environment...
   This will:
   1. Check for cached .npz (we just saw it loads instantly!)
   2. Check for cached .pkl (init states)
   3. If no cache: compute initial states for all windows
   4. Otherwise: load from cache (fast!)

self.datapath f:\OneDrive - 7t12kn\Documents\GitHub\ViT-MARL/AMZN/2012-06-21/
found 1 message files
found 1 book files
Loading cached arrays from f:\OneDrive - 7t12kn\Documents\GitHub\ViT-MARL\saved_npz/lobster_AMZN_2012-06-21_10_fixed_time_1800_60_100_34200_57600.npz
START:  pre-reset in the initialization
pre-reset will be saved to  f:\OneDrive - 7t12kn\Documents\GitHub\ViT-MARL/pre_reset_states/ResetState_window_resolution_60_eptype_fixed_time_depth_10_stock_AMZN_windowidx_-1_nMsgPerStep_100_episode_time_1800_TimePeriod_2012-06-21.pkl
LOADING STATES FROM PKL...
DONE: pre-reset in the initialization

✓ Environment created!
   Number of episode windows: 390
   Messages available: (267328, 8)
   Ini

In [8]:
print("##AI-generated placeholder")
# ═══════════════════════════════════════════════════════════════
# DEMONSTRATION: env.reset()
# ═══════════════════════════════════════════════════════════════

print("="*70)
print("CALLING env.reset()")
print("="*70)

# Get default parameters (contains the data we loaded earlier!)
env_params = env.default_params

print("\nenv_params contains:")
print(f"  message_data shape: {env_params.message_data.shape}")
print(f"  book_data shape: {env_params.book_data.shape}")
print(f"  init_states_array: {type(env_params.init_states_array)}")

# Reset the environment
print("\n🔄 Calling env.reset()...")
obs, state = env.reset(key_reset, env_params)

print("\n✓ Reset complete!")
print(f"\nState returned:")
print(f"  Window index: {state.window_index}")
print(f"  Init time: {state.init_time} (seconds)")
print(f"  Start index: {state.start_index} (where to fetch messages)")
print(f"  Step counter: {state.step_counter}")
print(f"  Max steps: {state.max_steps_in_episode}")

# Convert init_time to human-readable
init_seconds = state.init_time[0]
hours = init_seconds // 3600
minutes = (init_seconds % 3600) // 60
seconds = init_seconds % 60
print(f"\n  Episode starts at: {hours:02d}:{minutes:02d}:{seconds:02d}")

print(f"\nOrderbook state:")
print(f"  Ask orders shape: {state.ask_raw_orders.shape}")
print(f"  Bid orders shape: {state.bid_raw_orders.shape}")
print(f"  Trades shape: {state.trades.shape}")

print("\n" + "="*70)

##AI-generated placeholder
CALLING env.reset()

env_params contains:
  message_data shape: (267328, 8)
  book_data shape: (390, 40)
  init_states_array: <class 'gymnax_exchange.jaxen.StatesandParams.LoadedEnvState'>

🔄 Calling env.reset()...

✓ Reset complete!

State returned:
  Window index: 122
  Init time: [41520     0] (seconds)
  Start index: 92727 (where to fetch messages)
  Step counter: 0
  Max steps: 170

  Episode starts at: 11:32:00

Orderbook state:
  Ask orders shape: (100, 6)
  Bid orders shape: (100, 6)
  Trades shape: (100, 8)



In [9]:
print("##AI-generated placeholder")
# ═══════════════════════════════════════════════════════════════
# DEMONSTRATION: env.step()
# ═══════════════════════════════════════════════════════════════

print("="*70)
print("CALLING env.step()")
print("="*70)

# Create a dummy action (in base env, action doesn't matter)
action = {}

# Split key for randomness
rng, key_step = jax.random.split(rng, 2)

print("\n⚡ Taking step 1...")
print(f"   Current state: window_index={state.window_index}, step={state.step_counter}")

# What messages will be fetched?
msg_start = state.start_index + state.step_counter * env.n_data_msg_per_step
msg_end = msg_start + env.n_data_msg_per_step
print(f"   Will fetch messages: msgs[{msg_start}:{msg_end}]")

# Take the step!
obs, state, reward, done, info = env.step_env(key_step, state, action, env_params)

print(f"\n✓ Step complete!")
print(f"   New step counter: {state.step_counter}")
print(f"   Reward: {reward}")
print(f"   Done: {done}")

# Take a few more steps
print("\n⚡ Taking 4 more steps...")
for i in range(4):
    rng, key_step = jax.random.split(rng, 2)
    obs, state, reward, done, info = env.step_env(key_step, state, action, env_params)
    print(f"   Step {state.step_counter}: done={done}")
    
    if done:
        print(f"   Episode finished at step {state.step_counter}!")
        break

print("\n" + "="*70)
print("COMPLETE WORKFLOW DEMONSTRATION FINISHED!")
print("="*70)
print("\n📚 Summary of what we've seen:")
print("   1. ✓ Loaded data from .npz cache")
print("   2. ✓ Explored message and orderbook structure")
print("   3. ✓ Created BaseLOBEnv (loads init states)")
print("   4. ✓ Called env.reset() - picked a window instantly")
print("   5. ✓ Called env.step() - processed messages through orderbook")
print("\n🎯 This is the complete pipeline from CSV → Training!")
print("="*70)

##AI-generated placeholder
CALLING env.step()

⚡ Taking step 1...
   Current state: window_index=122, step=0
   Will fetch messages: msgs[92727:92827]

✓ Step complete!
   New step counter: 92727
   Reward: 0
   Done: False

⚡ Taking 4 more steps...
   Step 170: done=True
   Episode finished at step 170!

COMPLETE WORKFLOW DEMONSTRATION FINISHED!

📚 Summary of what we've seen:
   1. ✓ Loaded data from .npz cache
   2. ✓ Explored message and orderbook structure
   3. ✓ Created BaseLOBEnv (loads init states)
   4. ✓ Called env.reset() - picked a window instantly
   5. ✓ Called env.step() - processed messages through orderbook

🎯 This is the complete pipeline from CSV → Training!
